In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


+ Cho model học từ giao dịch bình thường => khi test fraud, nếu reconstruction error cao (dữ liệu được tạo ra bị lệch khỏi phân phối Gaussian quá nhiều) => "Bất thường"

+ Nếu các bạn dùng AE, thì fraud vẫn có thể được tại tạo tốt

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Đọc data
df = pd.read_csv("/content/drive/MyDrive/ML_DL_datasets/Synthetic_Financial_datasets_log.csv")
df = df.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1)

In [ ]:
# Phân loại normal và fraud
normal_df = df[df["isFraud"] == 0]
fraud_df = df[df["isFraud"] == 1]

In [ ]:
# Features
categorical_features = ['type']
numeric_features = ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']

In [ ]:
# Thiết lập pipeline preprocess data trước khi đưa vào model (xử lý cho normal_df)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)
    ]
)

In [ ]:
# Fit và transform normal_df
X_normal = preprocessor.fit_transform(normal_df.drop('isFraud', axis=1))

In [ ]:
# Transform toàn bộ để test
X_all = preprocessor.transform(df.drop('isFraud', axis=1))
labels = df["isFraud"].values

In [ ]:
# Convert to tensor
normal_tensor = torch.tensor(X_normal, dtype=torch.float32)
all_tensor = torch.tensor(X_all, dtype=torch.float32)

# Dataloader cho normal
dataset = TensorDataset(normal_tensor, normal_tensor)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x):
        h = torch.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, z):
        h = torch.relu(self.fc1(z))
        x_hat = torch.sigmoid(self.fc2(h))  # Nếu data không [0,1], thay bằng: x_hat = self.fc2(h)
        return x_hat

In [ ]:
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dim, input_dim)

    def forward(self, x):
        # Tính được mean của phân phối Gaussian (mu)
        mu, logvar = self.encoder(x)
        # Variance: phương sai (logvar * 0.5)
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        # eps là noise (đáp ứng được yêu cầu N(0,1))
        # N(0, 1) là ký hiệu toán học để chỉ phân phối Gaussian chuẩn (mean=0, std=1)
        z = mu + std * eps # latent vector
        x_hat = self.decoder(z)
        return x_hat, mu, logvar

In [ ]:
# Loss function
def vae_loss(recon_x, x, mu, logvar):
    # đo lường sự khác biệt giữa input gốc (x) và output tạo mới (recon_x)
    recon_loss = nn.functional.mse_loss(recon_x, x, reduction="sum")
    # KL loss (Kullback-Leibler divergence) như kiểm tra "bạn có khớp với đám đông không?"
    # N(0, 1) => tiêu chuẩn của đám đông => Kiểm tra z từ data có khớp với tiêu chuẩn không?
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kl_loss # tính tổng loss để optimizer minimize

In [ ]:
input_dim = X_normal.shape[1]
hidden_dim = 64
latent_dim = 4
epochs = 50

In [ ]:
model = VAE(input_dim, hidden_dim, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for data in dataloader:
        inputs, _ = data
        recon, mu, logvar = model(inputs)
        loss = vae_loss(recon, inputs, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(dataloader.dataset)
    print(f'Epoch {epoch+1}, Avg Loss: {avg_loss:.4f}')

Epoch 1, Avg Loss: 5.9940
Epoch 2, Avg Loss: 5.9475
Epoch 3, Avg Loss: 5.9429
Epoch 4, Avg Loss: 5.9375
Epoch 5, Avg Loss: 5.9335
Epoch 6, Avg Loss: 5.9365
Epoch 7, Avg Loss: 5.9309
Epoch 8, Avg Loss: 5.9300
Epoch 9, Avg Loss: 5.9291
Epoch 10, Avg Loss: 5.9279


In [ ]:
# Tính anomaly score
model.eval()
with torch.no_grad():
    recon_all, mu_all, logvar_all = model(all_tensor)
    recon_errors = torch.mean((recon_all - all_tensor)**2, dim=1)
    kl_divs = -0.5 * torch.sum(1 + logvar_all - mu_all.pow(2) - logvar_all.exp(), dim=1)
    scores = recon_errors + kl_divs  # Kết hợp
    scores = scores.numpy()

In [ ]:
normal_scores = scores[:len(normal_df)]
threshold = np.mean(normal_scores) + 3 * np.std(normal_scores)
print(f'Threshold: {threshold:.4f}')

In [ ]:
auc = roc_auc_score(labels, scores)
print(f'AUC Score: {auc:.4f}')